# topographic correlation analyses for full resting state recordings

PSDs were fit using regular linear regression, which means that increases and decreases in Theta will not be ignored.

In [1]:
import os
import glob
import datetime

import seaborn as sns
import numpy as np
import scipy as sp
import pandas as pd
import scipy.io
import numpy.fft
import scipy.signal
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sklearn import linear_model

%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)

In [2]:
df = pd.read_csv('../work/2016-08-28-18:34:41.350419/ya-oa-full-linreg-2-24-eyesc-eyeso.csv')
df.head()

,SUBJECT,CLASS,AGE,SEX,AVG_OA_PSD_EYESC,AVG_OA_PSD_EYESO,AVG_YA_PSD_EYESC,AVG_YA_PSD_EYESO,AVG_PSD_EYESC,AVG_PSD_EYESO,...,B26_EYESO,B27_EYESO,B28_EYESO,B29_EYESO,B30_EYESO,FRONTAL_EYESO,LTEMPORAL_EYESO,CENTRAL_EYESO,RTEMPORAL_EYESO,OCCIPITAL_EYESO
0,120127101,SA_Control,80.84873,2.0,-4.34227,-3.134892,-6.051054,-5.046973,-6.862338,-5.993324,...,-6.837667,-8.136131,-8.812830,-7.736430,-5.903613,-7.766009,-9.870920,-9.256354,-8.718649,-7.131687
1,120127102,MCI_Control,71.89870,1.0,-4.34227,-3.134892,-6.051054,-5.046973,-5.043768,-1.707319,...,-2.327233,-5.807363,-6.186407,-4.466992,-2.957174,-3.996478,-3.017757,-5.742492,-1.570847,-3.653080
2,120127103,MCI_Control,69.43463,2.0,-4.34227,-3.134892,-6.051054,-5.046973,-4.404398,-2.421660,...,-2.995247,-5.738814,-6.343399,-5.717602,-4.165686,-4.397806,-5.551192,-6.602679,-5.150955,-3.998930
3,120127104,SA_Control,83.61396,2.0,-4.34227,-3.134892,-6.051054,-5.046973,-4.147895,-2.305099,...,-6.331208,-5.995042,-5.725195,-5.606819,-6.502365,-4.478913,-5.451922,-4.705022,-4.571635,-5.189071
4,120127105,SA_Control,80.69541,2.0,-4.34227,-3.134892,-6.051054,-5.046973,-5.327249,-5.730176,...,-8.104647,-8.302540,-6.848196,-7.533960,-8.635785,-5.981445,-5.432904,-6.554409,-6.100613,-8.137746


In [3]:
ch_eyesc = list(df.columns.values)[10:63]
ch_eyeso = list(df.columns.values)[68:121]
print(len(ch_eyesc))
print(len(ch_eyeso))

53
53


In [4]:
oa_voytek = df[df.AGE >= 60]
oa_voytek = oa_voytek[oa_voytek.CLASS != 'SA']
ya_voytek = df[df.AGE <= 35.5]
voytek = pd.concat([oa_voytek, ya_voytek])
voytek.head()

,SUBJECT,CLASS,AGE,SEX,AVG_OA_PSD_EYESC,AVG_OA_PSD_EYESO,AVG_YA_PSD_EYESC,AVG_YA_PSD_EYESO,AVG_PSD_EYESC,AVG_PSD_EYESO,...,B26_EYESO,B27_EYESO,B28_EYESO,B29_EYESO,B30_EYESO,FRONTAL_EYESO,LTEMPORAL_EYESO,CENTRAL_EYESO,RTEMPORAL_EYESO,OCCIPITAL_EYESO
0,120127101,SA_Control,80.84873,2.0,-4.34227,-3.134892,-6.051054,-5.046973,-6.862338,-5.993324,...,-6.837667,-8.136131,-8.812830,-7.736430,-5.903613,-7.766009,-9.870920,-9.256354,-8.718649,-7.131687
1,120127102,MCI_Control,71.89870,1.0,-4.34227,-3.134892,-6.051054,-5.046973,-5.043768,-1.707319,...,-2.327233,-5.807363,-6.186407,-4.466992,-2.957174,-3.996478,-3.017757,-5.742492,-1.570847,-3.653080
2,120127103,MCI_Control,69.43463,2.0,-4.34227,-3.134892,-6.051054,-5.046973,-4.404398,-2.421660,...,-2.995247,-5.738814,-6.343399,-5.717602,-4.165686,-4.397806,-5.551192,-6.602679,-5.150955,-3.998930
3,120127104,SA_Control,83.61396,2.0,-4.34227,-3.134892,-6.051054,-5.046973,-4.147895,-2.305099,...,-6.331208,-5.995042,-5.725195,-5.606819,-6.502365,-4.478913,-5.451922,-4.705022,-4.571635,-5.189071
4,120127105,SA_Control,80.69541,2.0,-4.34227,-3.134892,-6.051054,-5.046973,-5.327249,-5.730176,...,-8.104647,-8.302540,-6.848196,-7.533960,-8.635785,-5.981445,-5.432904,-6.554409,-6.100613,-8.137746


In [5]:
r_values_eyesc = []
for ch in ch_eyesc:
    r_values_eyesc.append(np.corrcoef(voytek.AGE, voytek[ch])[0][1])
r_values_eyeso = []
for ch in ch_eyeso:
    r_values_eyeso.append(np.corrcoef(voytek.AGE, voytek[ch])[0][1])

In [6]:
r_values_eyesc

[-0.033519860684261667,
 0.0013277794263815653,
 0.11665022033375032,
 0.049888387221798232,
 0.080670839161046878,
 0.10760381945791674,
 0.062991703208144872,
 0.18620111314945795,
 0.17761760114929323,
 0.12987630736596567,
 0.014239914771475109,
 0.072688304096677647,
 0.14100560963495107,
 0.08539299530642315,
 0.013857690350544497,
 0.10405202376417111,
 0.046045194160651774,
 0.13044174973170386,
 0.091684732221011728,
 -0.036773191341443232,
 0.083648941449829942,
 0.011739956852954783,
 0.060836386861656898,
 0.16859330831425601,
 0.15244742025056204,
 0.11026075987369743,
 0.17888181498164463,
 0.10043361626441501,
 0.16605109959969394,
 0.11610325223119733,
 0.080813134713573043,
 0.050682551857192136,
 0.14395543838418629,
 0.18898345468495159,
 0.20238210909594592,
 0.039019187452618835,
 0.14082345300118665,
 0.05334775870601853,
 0.091748726810670186,
 0.17281168802902983,
 0.2070326158264458,
 0.12522615473213522,
 0.066090087822653354,
 -0.04334322021918223,
 0.0816381

In [7]:
r_values_eyeso

[0.30407632935627049,
 0.23307584634121747,
 0.21804535618265963,
 0.26355053424076785,
 0.33073267221227715,
 0.19562266302352962,
 0.1132185901000419,
 0.23995882558784398,
 0.17858804873638434,
 0.15778135991088132,
 -0.0140766292866488,
 0.082725867255556015,
 0.28902330463051962,
 0.27508640548822716,
 0.10539004578778043,
 0.12510396611341315,
 0.037183691871745764,
 0.21510996405858943,
 0.21972602584814091,
 0.1082096824282108,
 0.24862399734605245,
 0.17962906406636928,
 0.20158899551717527,
 0.26143126026025632,
 0.21186080253936757,
 0.29205388267751253,
 0.23270817246026493,
 0.38003192631294558,
 0.2542762442593921,
 0.35423644705510526,
 0.24219222389621867,
 0.17244174391063796,
 0.20982334093853486,
 0.076293721821333579,
 0.21313876326940864,
 0.13991759328355977,
 0.28267196380481863,
 0.16987876368599975,
 0.077169719850362195,
 0.098800415255503052,
 0.28055970543505737,
 0.19414290741833665,
 0.2170105778924028,
 0.13765475776479824,
 0.093653438116060139,
 0.22996